In [9]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

import os, time, asyncio, glob, csv
import logging
import sys

from dotenv import load_dotenv

In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [4]:
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
database = os.getenv("NEO4J_DATABASE", "neo4j")
AUTH = (username, password)


In [5]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

Connection established.


In [6]:
graph = Neo4jGraph()

/var/folders/qs/wv32_qr17mncyfb4d22gpl640000gp/T/ipykernel_28471/267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [ ]:
# file_path = "data/Game of Thrones season 1 - Wikipedia.pdf"
# loader = PyPDFLoader(file_path=file_path)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [27]:
file_path = "data/dummytext.txt"
loader = TextLoader(file_path=file_path)
docs = loader.load()

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
documents = text_splitter.split_documents(documents=docs)

In [ ]:
llm_type = os.getenv("LLM_TYPE", "ollama")
if llm_type == "ollama":
    llm = ChatOllama(model="llama3.1:8b", temperature=0)
else:
    llm = None

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

/var/folders/qs/wv32_qr17mncyfb4d22gpl640000gp/T/ipykernel_28471/3585865647.py:2: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1:8b", temperature=0, format="json")


In [39]:
entities = [
    {"label": "Person", "properties": [{"name": "name", "type": "STRING"}]},
    {"label": "House", "properties": [{"name": "name", "type": "STRING"}]},
    {"label": "Planet", "properties": [{"name": "name", "type": "STRING"}]}
]

relations = [
    {"label": "PARENT_OF", "source": "Person", "target": "Person"},
    {"label": "HEIR_OF", "source": "Person", "target": "House"},
    {"label": "RULES", "source": "House", "target": "Planet"},
]

In [45]:
pipeline = SimpleKGPipeline(
    llm=llm,
    driver=driver,
    embedder=embedder,
    entities=entities,
    relations=relations,
    on_error="IGNORE",
)

PipelineDefinitionError: 